# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Hashtable Chaining

In [1]:
import time

In [2]:
from types import SimpleNamespace
import numpy as np

# Algorithm

In [3]:
%%time
class HashTable:

    ratio_expand = .8
    ratio_shrink = .2
    min_size = 11

    def __init__(self, size=None):
        self._size = size or self.min_size
        self._buckets = [None] * self._size
        self._list = None
        self._count = 0

    def _entry(self, key):
        # get hash and index
        idx = hash(key) % self._size

        # find entry by key
        p, q = self._buckets[idx], None
        while p and p.key != key:
            p, q = p.next, p

        # index, entry, previous entry
        return idx, p, q

    def _ensure_capacity(self):
        fill = self._count / self._size

        # expand or shrink?
        if fill > self.ratio_expand:
            self._size = self._size * 2 + 1
        elif fill < self.ratio_shrink and self._size > self.min_size:
            self._size = (self._size - 1) // 2
        else:
            return

        # reallocate buckets
        self._buckets = [None] * self._size

        # store entries into new buckets
        p = self._list
        while p:
            idx = hash(p.key) % self._size
            p.next = self._buckets[idx]
            self._buckets[idx] = p
            p = p.entry_next

    def __len__(self):
        return self._count

    def __contains__(self, key):
        _, p, _ = self._entry(key)
        return bool(p)

    def __getitem__(self, key):
        _, p, _ = self._entry(key)
        return p and p.value

    def __setitem__(self, key, value):
        idx, p, _ = self._entry(key)

        # set entry if key was found
        if p:
            p.value = value
            return

        # create new entry
        p = SimpleNamespace(
            key=key,
            value=value,
            next=self._buckets[idx],
            entry_next=self._list,
            entry_prev=None
        )

        # store to bucket
        self._buckets[idx] = p

        # store to list
        if self._list:
            self._list.entry_prev = p
        self._list = p

        # expand
        self._count += 1
        self._ensure_capacity()

    def __delitem__(self, key):
        idx, p, q = self._entry(key)

        # key not found
        if not p:
            return

        # remove from bucket
        if q:
            q.next = p.next
        else:
            self._buckets[idx] = p.next

        # remove from list
        if p.entry_next:
            p.entry_next.entry_prev = p.entry_prev
        if p.entry_prev:
            p.entry_prev.entry_next = p.entry_next
        else:
            self._list = p.entry_next

        # shrink
        self._count -= 1
        self._ensure_capacity()

    def __iter__(self):
        p = self._list
        while p:
            yield p.key
            p = p.entry_next

    def slots(self):
        return ''.join(p and 'x' or '-' for p in self._buckets)

CPU times: user 58 µs, sys: 0 ns, total: 58 µs
Wall time: 73.7 µs


# Run

In [4]:
%%time
table = HashTable()

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 53.6 µs


In [5]:
%%time
# add random values
for _ in range(1000):
    key, value = np.random.randint(1000), np.random.rand()
    table[key] = value

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 16.1 ms


In [6]:
%%time
len(table), table._size

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 20 µs


(622, 1535)

In [7]:
%%time
table.slots()

CPU times: user 0 ns, sys: 263 µs, total: 263 µs
Wall time: 270 µs


'x-xx-x-xxxx-x-x-------x-x-xx-xxx-xxxx-x-x---x--xx-xxxxx-x---x-x-xx-xx-xxxx-xxxx------x-xx-x--xx-xxxxx--xxx-xxxxxxxxx-x--xxxx-x---x-xxxx-------xx-xxx-----x-xxxx-xx------x-xxx--xxx--x--xxxx--xxxxxxxx--xxx--xxx-xx-xx-x-xx-xxxxx---xx-x--xxxx-xxxxxxx-xxx--xxxx-x-xx--xx----x-xx-xxx---x--x-xxxx---xxx-xx-x--------x-xx---x-x--x--xxxx-x-xxxxxxx-x-xxxx---x---xx---xxxxxxx-xx--x-xxx-xx-xxxxxx-xx-x-x--x----x-x-xx-xxxxxx--x---xxxx--x--xx----xxxxxxxx-xx-xxxx--xx-x-x-xxx--xxxxxx--xx---x--x--xxx-xx--xxx-xxxxxxxxx--xxx-xxx-xx-xx--x-xxx-x--x-x-xx-xxxx-xx-x----x-xx-xxxxx-xxxxxxx-xx---xxxxx-x-xxxxxxxxxx-x----xxxxxxxxx--x-xx--x-xxx-xx-xx-xxx--x--x-x---xxxx--xxx--xxxxx--xxxx-xxxx-x-xxxxx-xxx-xx-xxxxx-x----x--x-xxx-x--xx-xxxxxx-xxx-xxxxxx--xxx-xxxxxxx-x-x----xx--xxxxxxxxx-x-x-xxx-xxxxxxxxx-xx-xxxxxxxx-xxx---xx-x-xxxxxx-x-xx--x-x-xx-xxx--x--x-x-xxxx-xxx-xx----xx-x-x-xx---xxx-xxxx--x-x-xxx-xx--x--xxxx-x---xxxxxx-x-xxx--xx--xxxxxxxx-x-x-xxx--xx-xxx-xx-xxxxxxxxxx-xx-xx-xxx-x-xxxxx-xx-x-xxxx-xx-x-x--xx-xxxxxx-x-xx

In [8]:
%%time
# print some values
for key in list(table)[:5]:
    print(key, table[key])

112 0.7852329580360887
65 0.11276247710084486
24 0.3837493478967666
751 0.19456516600710383
572 0.43671517411873306
CPU times: user 449 µs, sys: 0 ns, total: 449 µs
Wall time: 441 µs


In [9]:
%%time
# delete all the values
for key in list(table):
    del table[key]

CPU times: user 3.27 ms, sys: 69 µs, total: 3.34 ms
Wall time: 7.53 ms


In [10]:
%%time
len(table), table._size

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.2 µs


(0, 11)

# The End